# Refresher on Binomial Pricing Model

Before jumping right into the Black-Scholes model I believe that it's important to describe what is going on under the hood. This can be done by first exploring the binomial pricing model for its simplicity in roughly describing what the Black-Scholes better achieves.  


# Why explore this? 

This is the most rudimentary option pricing model out there. It seeks to measure the price of an option by describing its characteristics using only two different payoff states each with their own probability of occuring. More on that in a bit. In short the binomial pricing model is useful in demonstrating how to approximate an option price, how to calculate risk-neutral probabilities, and illustrates the logic needed in approximating solutions where no explicit analytical solution exists (like for American Puts).
### A note:
The main example I will be using can be found in *An Introduction to Derivative Securities, Financial Markets, and Risk Management* by Robert Jarrow and Arkadev Chatterjea. I'm not going to show the binomial tree diagrams (that might change in the future). If you are a visual learner I highly recommend you attempt to draw your own to build familiarity. [Here](https://xplaind.com/552187/binomial-options-pricing-model) is a short guide showing how the numerical values interact. Additionally,  Investopedia is a good place to start as they show how to visualize multi-period binomial tree diagrams in Excel which can be found [here](https://www.investopedia.com/terms/b/binomialoptionpricing.asp). 

# A single period problem walkthrough

You go to look up Your Beloved Machine Inc (YBM) stock price and find it sitting exactly at `$100`. Suppose you have an accurate but broken crystal ball that lets you see two different futures 1 year from now with their own discrete probabilites of occuring (don't ask why...). In one future you see the stock price rise to `$120` and your crystal ball says that the probability of this happening is `3/4` chance. In the other future you see the stock price drop to `$90.25` with a probaiblity of `1/4`. You know for certain it will be one or the other. According to the crystal ball one of these realities must occur. If its not `$120` then it will be `$90.25` It can't be any other number. Let's refer to these as the `up` and `down` states.

* Price today: 100
* After 1 year 2 different possibilties:


|Outcome|Price|   Probability|
|-------|-----|-----------|
|Up|120.00|75%|
|Down|90.25|25%|

You want to try your hand at buying a European style call option for YBM stock. You get the option info and it says the strike price is `$110` and the cost of the call option is `$4.76`. In addition, you know the risk-free rate is 5.13% (wow, what a rate!). Now you are an intelligent investor and want to make sure you are getting a good deal. The trouble is, you have no idea if the premium  for the option is a resonable price! The more you think about it, the more you wonder how a person comes up with this price. So you open up your derivatives textbook and get to work figuring out how to calculate the price on your own.

You recall the no-arbitrage rule and assume that the options follows a similiar rule (not going to comment on that loaded assumption). You figure that by replicating the call option using other assets and seeing what it costs then by the no-arbitrage principal that will be the call option's premium. You set about building a $synthetic$ $call$. 

# Still need to finish transcribing my notes to this notebook...

# But, here's the final product of my notes:

In [7]:
# the final product

class BinomialOption:
    def __init__(self, stock_price, strike_price, up_value, down_value, risk_free, option_type = 'Call'):
        self.stock_price = stock_price
        self.strike_price = strike_price
        self.up_value = up_value
        self.down_value = down_value
        self.risk_free = risk_free
        self.option_type = option_type
        
        if option_type == 'Call':
            self.option_is_a_put = False
        if option_type == 'Put':
            self.option_is_a_put = True 

        
        # calculations done all at once.
        
        # flow control for call or put options
        # in the up state
        if self.option_is_a_put:
            # this mimics the up state payoff structure for a put 
            self.up_payoff = max(self.strike_price - self.up_value, 0)
        else:
            # this mimics the up payoff structure for a call 
            self.up_payoff = max(self.up_value - self.strike_price, 0) 
        
        # in the down state
        if self.option_is_a_put:
            # this mimics the down payoff structure for a put 
            self.down_payoff = max(self.strike_price - self.down_value, 0)
        else: 
            self.down_payoff = max(self.down_value - self.strike_price, 0)
        

        # 
        self.hedge_ratio =  (self.up_payoff - self.down_payoff)/(self.up_value - self.down_value)
        self.rf_units = (1/(1+self.risk_free))*(self.up_payoff-(self.hedge_ratio*self.up_value))

        #work back to present value. By no-arbitrage rules and replication this calculation is the option price. 
        self.option_price = self.hedge_ratio*self.stock_price + self.rf_units
        
        #risk neutral probabilites
        self.up_risk_neutral_prob = ((1+self.risk_free)*(self.stock_price)-self.down_value)/(self.up_value-self.down_value)
        self.down_risk_neutral_prob = 1-self.up_risk_neutral_prob

    # packed into a nice display function    
    def print_calc_values(self, rounding = 2, hide_hegde_ratio = False, hide_risk_free_units = False, hide_state_payoffs = False, hide_risk_neutral_probabilites = False):
        
        # provides a message to user if any field is hidden.
        if hide_hegde_ratio or hide_risk_free_units or hide_state_payoffs or hide_risk_neutral_probabilites:
            headsup = 'Additionally, NOT ALL FIELDS ARE DISPLAYED!'
        else:
            headsup = ''
        print('''The following values are for a single period {} option where the underlying value is {}, a strike price of {}, an up value of {}, 
a down value of {}, and a risk free rate of {}%. All outputs are rounded to {} decimal places. {}'''.format(self.option_type, self.stock_price, self.strike_price, self.up_value, self.down_value, self.risk_free, rounding, headsup))

        print('''\n------------------------
{} Option Information
------------------------'''.format(self.option_type))
        
        print('Calculated Option Price: {}'.format(round(self.option_price, rounding)))
        
        if hide_hegde_ratio:
            pass
        else:
            print('Calculated Hedge Ratio: {}'.format(round(self.hedge_ratio, rounding)))

        if hide_risk_free_units:
            pass
        else:
            print('Calculated present value of bond position: {}'.format(round(self.rf_units, rounding)))

        if hide_state_payoffs:
            pass
        else:
            print('Up state payoff is {} and down state payoff is {}'.format(self.up_payoff, self.down_payoff))

        if hide_risk_neutral_probabilites:
            pass
        else: 
            print('Calculated risk neutral probability for up state is {} and for down state risk neutral probability is {}'.format(round(self.up_risk_neutral_prob, rounding), round(self.down_risk_neutral_prob, rounding)))

# Testing the class. Try a call option

In [8]:
# start with a call option (by default the program assumes a call option)
b_call = BinomialOption(100, 110, 120, 90.25, 0.0513)

# by default display all relavent information. Defaults to rounding to 2 decimal places.
b_call.print_calc_values()

The following values are for a single period Call option where the underlying value is 100, a strike price of 110, an up value of 120, 
a down value of 90.25, and a risk free rate of 0.0513%. All outputs are rounded to 2 decimal places. 

------------------------
Call Option Information
------------------------
Calculated Option Price: 4.76
Calculated Hedge Ratio: 0.34
Calculated present value of bond position: -28.86
Up state payoff is 10 and down state payoff is 0
Calculated risk neutral probability for up state is 0.5 and for down state risk neutral probability is 0.5


# Try a put option now:


In [9]:
b_put = BinomialOption(100, 110, 120, 90.25, 0.0513, 'Put')
b_put.print_calc_values()

The following values are for a single period Put option where the underlying value is 100, a strike price of 110, an up value of 120, 
a down value of 90.25, and a risk free rate of 0.0513%. All outputs are rounded to 2 decimal places. 

------------------------
Put Option Information
------------------------
Calculated Option Price: 9.39
Calculated Hedge Ratio: -0.66
Calculated present value of bond position: 75.78
Up state payoff is 0 and down state payoff is 19.75
Calculated risk neutral probability for up state is 0.5 and for down state risk neutral probability is 0.5


# Can change the level of detail by providing a rounding argument:

In [10]:
b_put = BinomialOption(100, 110, 120, 90.25, 0.0513, 'Put')
b_put.print_calc_values(rounding = 4)

The following values are for a single period Put option where the underlying value is 100, a strike price of 110, an up value of 120, 
a down value of 90.25, and a risk free rate of 0.0513%. All outputs are rounded to 4 decimal places. 

------------------------
Put Option Information
------------------------
Calculated Option Price: 9.39
Calculated Hedge Ratio: -0.6639
Calculated present value of bond position: 75.7765
Up state payoff is 0 and down state payoff is 19.75
Calculated risk neutral probability for up state is 0.5002 and for down state risk neutral probability is 0.4998


### Notice that the risk-neutral probabilites are not actually 0.5 but slightly different.

### Also, notice that the price option still rounded. This is because it's extremely close to `$9.39`. Observe the following:

In [11]:
# same as before with slightly different formating.
b_put.print_calc_values(rounding = 6)

The following values are for a single period Put option where the underlying value is 100, a strike price of 110, an up value of 120, 
a down value of 90.25, and a risk free rate of 0.0513%. All outputs are rounded to 6 decimal places. 

------------------------
Put Option Information
------------------------
Calculated Option Price: 9.389975
Calculated Hedge Ratio: -0.663866
Calculated present value of bond position: 75.77653
Up state payoff is 0 and down state payoff is 19.75
Calculated risk neutral probability for up state is 0.500168 and for down state risk neutral probability is 0.499832


# Can also hide fields

In [12]:
# This should only display option price, which is always displayed.
b_put.print_calc_values(hide_hegde_ratio = True, hide_risk_free_units=True, hide_state_payoffs=True, hide_risk_neutral_probabilites=True)

The following values are for a single period Put option where the underlying value is 100, a strike price of 110, an up value of 120, 
a down value of 90.25, and a risk free rate of 0.0513%. All outputs are rounded to 2 decimal places. Additionally, NOT ALL FIELDS ARE DISPLAYED!

------------------------
Put Option Information
------------------------
Calculated Option Price: 9.39


### Notice that hiding any field will prompt a warning that not all fields are being displayed.

In [13]:
# opt to hide state payoffs
b_put.print_calc_values(hide_state_payoffs=True)

The following values are for a single period Put option where the underlying value is 100, a strike price of 110, an up value of 120, 
a down value of 90.25, and a risk free rate of 0.0513%. All outputs are rounded to 2 decimal places. Additionally, NOT ALL FIELDS ARE DISPLAYED!

------------------------
Put Option Information
------------------------
Calculated Option Price: 9.39
Calculated Hedge Ratio: -0.66
Calculated present value of bond position: 75.78
Calculated risk neutral probability for up state is 0.5 and for down state risk neutral probability is 0.5


# A couple more examples

In [14]:
another_call_option = BinomialOption(60, 61, 66, 57, 0.05)
another_call_option.print_calc_values()

The following values are for a single period Call option where the underlying value is 60, a strike price of 61, an up value of 66, 
a down value of 57, and a risk free rate of 0.05%. All outputs are rounded to 2 decimal places. 

------------------------
Call Option Information
------------------------
Calculated Option Price: 3.17
Calculated Hedge Ratio: 0.56
Calculated present value of bond position: -30.16
Up state payoff is 5 and down state payoff is 0
Calculated risk neutral probability for up state is 0.67 and for down state risk neutral probability is 0.33


### That looks correct... try the same thing but make it a put this time

In [15]:
another_put_option = BinomialOption(60, 61, 66, 57, 0.05, 'Put')
another_put_option.print_calc_values()

The following values are for a single period Put option where the underlying value is 60, a strike price of 61, an up value of 66, 
a down value of 57, and a risk free rate of 0.05%. All outputs are rounded to 2 decimal places. 

------------------------
Put Option Information
------------------------
Calculated Option Price: 1.27
Calculated Hedge Ratio: -0.44
Calculated present value of bond position: 27.94
Up state payoff is 0 and down state payoff is 4
Calculated risk neutral probability for up state is 0.67 and for down state risk neutral probability is 0.33


# Again that looks right, but couldn't hurt to double check. Does it pass the put-call parity test?

That is, does the following hold?

$\begin{equation*} 
stock + put = PV(strike) + call
\end{equation*}$

From above stock = `60`, put = `1.27`, strike = `61`, call is `3.17`, and interest rate is  `0.05`. 
So,

$\begin{equation*} 
60 + 1.27 = PV(61) + 3.17\\
\end{equation*}$

$\begin{equation*} 
61.27 = \frac{61}{1+R} + 3.17
\end{equation*}$

$\begin{equation*} 
61.27 = \frac{61}{1+0.05} + 3.17
\end{equation*}$

$\begin{equation*} 
61.27 = 61.27
\end{equation*}$

The put-call parity test holds. 

In [22]:
# extra stuff
left_side = 60 + another_put_option.option_price
right_side = 61/1.05 + another_call_option.option_price

print(left_side)
print(right_side)

61.269841269841265
61.269841269841265


# Everything checks out

I dropped the class code into its own file called `options.py` Below shows a simple use of the module. I have another notebook to see the most up to date usage of the options.py file. Go check it out

In [19]:
from allthingsoptions import options

In [20]:
b_call_test = options.BinomialOption(100, 110, 120, 90.25, 0.0513)

b_call_test.print_calc_values()

The following values are for a single period Call option where the underlying value is 100, a strike price of 110, an up value of 120, 
a down value of 90.25, and a risk free rate of 0.0513%. All outputs are rounded to 2 decimal places. 

------------------------
Call Option Information
------------------------
Calculated Option Price: 4.76
Calculated Hedge Ratio: 0.34
Calculated present value of bond position: -28.86
Up state payoff is 10 and down state payoff is 0
Calculated risk neutral probability for up state is 0.5 and for down state risk neutral probability is 0.5


In [21]:
single_period_call = options.BinomialOption(100, 110, 120, 90, 0.10, 'Call')
single_period_call.print_calc_values()

The following values are for a single period Call option where the underlying value is 100, a strike price of 110, an up value of 120, 
a down value of 90, and a risk free rate of 0.1%. All outputs are rounded to 2 decimal places. 

------------------------
Call Option Information
------------------------
Calculated Option Price: 6.06
Calculated Hedge Ratio: 0.33
Calculated present value of bond position: -27.27
Up state payoff is 10 and down state payoff is 0
Calculated risk neutral probability for up state is 0.67 and for down state risk neutral probability is 0.33


In [9]:
30/1.1

27.27272727272727